## Exposing your tools from the MCP server

If you want to expose your tools or data so that other agents would be able to access it, then you could leverage MCP servers. Databricks already provides [managed MCP servers](https://docs.databricks.com/aws/en/generative-ai/mcp/managed-mcp#available-managed-servers), so that you do not need to start from scratch. 


These managed servers include the UC functions, vector search indexes, and Genie spaces. In our example, we will show how you can provide the agent, the UC functions that we have created in [create_tools](https://adb-984752964297111.11.azuredatabricks.net/editor/notebooks/2254899697811791?o=984752964297111#command/6040444310329070) and the vector search indexes from [pdf-rag-tool](https://adb-984752964297111.11.azuredatabricks.net/editor/notebooks/2254899697812000?o=984752964297111#command/6040444310325373) as tools stored in Databricks managed MCP servers. 

In [0]:
%pip install -U -qqqq mlflow>=3.1.1 langchain langgraph databricks-langchain pydantic databricks-agents databricks-mcp  unitycatalog-langchain[databricks] uv databricks-feature-engineering==0.12.1
dbutils.library.restartPython()

In [0]:
%run ../_resources/01-setup

## Databricks Managed MCP

To use the UC functions and Vector Search managed MCP servers, we'd just need to add in our **agent.py** code the relevant URLs for both of them:

- **Vector search**: https://<workspace-hostname>/api/2.0/mcp/vector-search/{catalog}/{schema}
- **Unity Catalog functions**: https://<workspace-hostname>/api/2.0/mcp/functions/{catalog}/{schema}

## Update our existing Agent to add the MCP servers tools

Now that our functions and vector search indexes ready, all we have to do is to add it the managed MCP servers for both UC functions and Vector Search to our existing agent!

We'll reuse the `agent.py` and `agent_config.yaml` file: simply add the MCP configuration and our agent will add it as one of the tools available!

In [0]:
import mlflow
import yaml, sys, os
import mlflow.models

# Locate the agent_eval folder & config
agent_eval_path = os.path.abspath(os.path.join(os.getcwd(), "../02-agent-eval"))
sys.path.append(agent_eval_path)
mlflow.set_experiment(agent_eval_path + "/02.1_agent_evaluation")
conf_path = os.path.join(agent_eval_path, 'agent_config.yaml')

try:
    config = yaml.safe_load(open(conf_path))
    config["config_version_name"] = "model_with_retriever_and_mcp"


    yaml.dump(config, open(conf_path, "w"))
except Exception as e:
    print(f"Skipped update - ignore for job run - {e}")

model_config = mlflow.models.ModelConfig(development_config=conf_path)

## Test the agent

In [0]:
print("Loaded mcp_server_urls:", model_config.get("mcp_server_urls"))


In [0]:
from agent import MinimalMCPAgent

MCP_SERVERS = ["https://adb-984752964297111.11.azuredatabricks.net/api/2.0/mcp/functions/system/ai"]  # replace with real MCP 
agent = MinimalMCPAgent(MCP_SERVERS)
print(agent.list_tools())

In [0]:
from agent import AGENT
request_example = "List all tools"
answer = AGENT.predict({"input":[{"role":"user","content":request_example}]})
print(answer)

In [0]:
from mlflow.types.responses import ResponsesAgentRequest

request_example = "List all available tools you can use."
answer = AGENT.predict(
    ResponsesAgentRequest(
        input=[{"role": "user", "content": request_example}]
    )
)
print(answer)


In [0]:
with mlflow.start_run(run_name=model_config.get('config_version_name')):
  logged_agent_info = mlflow.pyfunc.log_model(
    name="agent",
    python_model=agent_eval_path+"/agent.py",
    model_config=conf_path,
    input_example={"input": [{"role": "user", "content": request_example}]},
     # Determine resources (endpoints, fonctions, vs...) to specify for automatic auth passthrough for deployment
    resources=AGENT.get_resources(),
    extra_pip_requirements=["databricks-connect"]
    )

In [0]:
from agent import AGENT, MCPToolWrapper

print("=== All tools loaded ===")
for t in AGENT.tools:
    print(f"{type(t)} - {t.name}")

print("\n=== MCP tools only ===")
for t in AGENT.tools:
    if isinstance(t, MCPToolWrapper):
        print(f"- {t.name}: {t.description} (from {t.server_url})")

In [0]:
from mlflow.types.responses import ResponsesAgentRequest

request = ResponsesAgentRequest(
    input=[{"role": "user", "content": "Run Python code: print(2 + 2)"}]
)

response = AGENT.predict(request)
print("Final output:", response.output)

In [0]:
from agent import model_config
print("mcp_server_urls in config:", model_config.get("mcp_server_urls"))